## Training.ipynb
 In this notebook mainly different functions are tested. Also the adapted datasets are also generated here.
 In the following cells dataloader , training loop and additional functions are defined

In [ ]:
import torch
from tqdm import tqdm
from model import seedformer_dim128
from torch.utils.data import Dataset,DataLoader
import open3d as o3d
import os
from pytorch3d.loss import chamfer_distance
import numpy as np
from torch.optim.lr_scheduler import StepLR
import utils.utils as utils
from utils.utils import get_loss
from torch.optim.lr_scheduler import ReduceLROnPlateau
import logging

### Dataset and Dataloader

https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

Below is the implementation of custom dataset/dataloader. Additionally there is a collate function. The number of points in eaach pointcloud is 
different. In order to make them equal collate function is defined.

In [ ]:
class RacingDataset(Dataset):
    def __init__(self,root_dir):#4731
        self.root_dir = root_dir
        self.file_list = os.listdir(root_dir)
        self.filter_file_list = self.filter_list()
       # self.target_points = target_points

    def __len__(self):
        return len(self.filter_file_list)

    def __getitem__(self,index):
        pcd_path = os.path.join(self.root_dir,self.filter_file_list[index])
        pcd = o3d.io.read_point_cloud(pcd_path)

        points = torch.tensor(pcd.points, dtype=torch.float32)

        return points,pcd_path

   
    def filter_list(self):
        '''
        Filter the inputs so that only pcds with more than 50 points are included in the training
        :return:
        '''
        filtered_list=[]
        for filename in self.file_list:
            pcd = o3d.io.read_point_cloud(os.path.join(self.root_dir,filename))
            points = torch.tensor(pcd.points, dtype=torch.float32)
            if len(points)>=0:
                filtered_list.append(filename)
        return filtered_list
   

# Load the largest point cloud
pcd_pad = o3d.io.read_point_cloud("/home/omar/TUM/Data/cropped/sim/018840.pcd")
pcd_pad_tens = torch.tensor(pcd_pad.points, dtype=torch.float32)
#print(len(pcd_pad_tens))
# Create the dataset and dataloader
dataset = RacingDataset(root_dir="/home/omar/TUM/Data/cropped/real")
dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=8,collate_fn=utils.collate_fn)

print(len(dataloader))

In [ ]:
print(torch.__version__)
torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Training
This Python script trains a neural network model using SeedFormer for reconstructing 3D point clouds. Key functionalities include:
* Model Setup: Initializes and configures a PointNet++ model (seedformer_dim128) for point cloud reconstruction.

* Data Handling: Loads a custom dataset (RacingDataset) and sets up a DataLoader for efficient batch processing.

* Training Loop: Iterates through epochs and batches, computing reconstruction losses using Chamfer Distance. Optimizes model parameters using Adam optimizer with learning rate scheduling.

* Checkpoint Management: Saves model checkpoints based on validation loss improvements to facilitate resumable training and model evaluation.

* Logging and Monitoring: Tracks training progress with average epoch losses and updates a progress bar (tqdm) to visualize training status.

* This script enables training of a deep learning model for reconstructing point clouds, showcasing foundational practices in neural network training and evaluation.
SeedFormer: https://arxiv.org/pdf/2207.10315

In [ ]:
torch.manual_seed(42)
#del model
# Initialize your model
model = seedformer_dim128(up_factors=[1, 2, 4, 4])  # 512 1024 4096 9192

# Define device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logging.info(f"Using device: {device}")
model.to(device)

# Define DataLoader
dataset = RacingDataset(root_dir="/dev/shm/IAC/SeedFormer_2602_npy/cropped/real")
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=8, collate_fn=utils.collate_fn)

# Define optimizer and scheduler
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=3e-4)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=10)
epochs=200
# Define paths for checkpoint
checkpoint_path = 'training2205v002_pad3000_knn_big.pth'

# Check if a checkpoint exists
if os.path.exists(checkpoint_path):
    # Load the checkpoint
    checkpoint = torch.load(checkpoint_path)
    start_epoch = checkpoint['epoch'] + 1
    best_loss = checkpoint['loss']
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    logging.info(f"Checkpoint loaded. Resuming training from epoch {start_epoch}. Best loss so far: {best_loss}")
else:
    start_epoch = 0
    best_loss = float('inf')
    logging.info("No checkpoint found. Starting training from scratch.")
    logging.info(model)

# Training loop
train_losses = []
for epoch in range(start_epoch, epochs):
    running_loss = 0

    # Wrap the DataLoader with tqdm to track progress
    with tqdm(enumerate(dataloader, 0), total=len(dataloader), desc=f'Epoch {epoch + 1}/{epochs}', unit='batch') as pbar:
        for i, data in pbar:
            inputs, labels = data
            inputs = inputs.to(device)  # Move data to GPU if available

            optimizer.zero_grad()

            # Apply the model to the entire batch
            outputs = model(inputs)

            losses = []
            for input_pc, output_pc in zip(inputs, outputs[-1]):
                loss, _ = chamfer_distance(input_pc.unsqueeze(0), output_pc.unsqueeze(0))
                losses.append(loss * 1e3)

            loss = torch.mean(torch.stack(losses))
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix(loss=running_loss / (i + 1))  # Update tqdm progress bar with the current loss

            # Compute losses
            # loss_total, losses, gts = get_loss(outputs, inputs, inputs, sqrt=False)
            # Backpropagation and optimization
            # loss_total.backward()
            # optimizer.step()
            # Log the loss for monitoring
            # running_loss += loss_total.item()
            # cd_pc_item = losses[0].item()
            # total_cd_pc += cd_pc_item
            # cd_p1_item = losses[1].item()
            # total_cd_p1 += cd_p1_item
            # cd_p2_item = losses[2].item()
            # total_cd_p2 += cd_p2_item
            # cd_p3_item = losses[3].item()
            # total_cd_p3 += cd_p3_item
            # partial_item = losses[4].item()
            # total_partial += partial_item
            # Compute average losses for the batch
            # avg_cdc = total_cd_pc / len(inputs)
            # avg_cd1 = total_cd_p1 / len(inputs)
            # avg_cd2 = total_cd_p2 / len(inputs)
            # avg_cd3 = total_cd_p3 / len(inputs)
            # avg_partial = total_partial / len(inputs)
            # Update running loss
            # print("len inputs" , len(inputs))
            # print("loss ",loss_total/len(inputs))
            # Update progress bar with the current loss

        avg_loss = running_loss / len(dataloader)
        scheduler.step(avg_loss)  # Use the average loss for the scheduler

    # Compute average loss for the epoch
    avg_loss = running_loss / len(dataloader)
    train_losses.append(avg_loss)

    # Save checkpoint if current loss is the best seen so far
    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'loss': avg_loss
        }, checkpoint_path)

    # Log the epoch loss
    last_lr = optimizer.param_groups[0]['lr']  # Get the last learning rate
    # logging.info(   f'Epoch {epoch + 1} Loss: {loss_total, avg_cd1, avg_cd2, avg_cd3, avg_partial} LR: {last_lr}')  # Log it into the file
    logging.info(f'Epoch {epoch + 1} Loss: {avg_loss} LR: {last_lr}')
    train_losses.append(avg_loss)

logging.log(train_losses)
print('Finished Training')


 ## Save The Data
 Below code generates adapted datase using the trained model and applying it to the sim data

In [ ]:
simulation_dataset = RacingDataset(root_dir="/home/omar/TUM/Data/cropped/sim")
simulation_dataloader = DataLoader(simulation_dataset, batch_size=1, shuffle=True, num_workers=8,collate_fn=utils.collate_fn)
utils.apply_and_save_res(dataset=simulation_dataset,dataloader=simulation_dataloader,model=model,savedir="/home/omar/TUM/Data/reconstructed_cropped/sim_2205v002_pad3000_knn_big")

In [ ]:
pcd=o3d.io.read_point_cloud("/home/omar/TUM/Data/reconstructed_cropped/sim_2205v002_pad3000_knn_big/018845.pcd")
print(len(pcd.points))
box=["Car" ,0.0, 0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,1.18, 1.9 ,4.88 ,26.11 ,-5.9 ,0.59 ,0.02]
utils.visualize(pcl=pcd,bbox_coordinates=box)

## "Stitch" the data
the generated car point clouds are then copied back to the original simulation scene.

In [ ]:
reconstructed_car_path="/home/omar/TUM/Data/reconstructed_cropped/sim_2205v002_pad3000_knn_big/"
original_sim_path="/home/omar/TUM/data_MA/m1695833/Sim2RealDistributionAlignedDataset/sim/data/pcl/"
bbox_path="/home/omar/TUM/data_MA/m1695833/Sim2RealDistributionAlignedDataset/sim/data/label"

npy_final_path="/home/omar/TUM/Data/SeedFormer_2602_npy/reconstructed_2205v002_pad3000_knn_big/points/"
file_car_list=[]
for filename_car in os.listdir(reconstructed_car_path):
    file_car_list.append(filename_car)
    txt=filename_car.replace('.pcd',".txt")
    bbox = (open(bbox_path+"/"+txt, "r")).read().split()
    original_pcd = o3d.io.read_point_cloud(original_sim_path+filename_car)
    filename_=filename_car.split(".")[0]

    reconstructed_car= o3d.io.read_point_cloud(reconstructed_car_path +"/"+ filename_car)
    crop_invert=utils.crop_invert_stitch(original_pcd,reconstructed_car, bbox)
    np_array=np.asarray(crop_invert.points)
    np.save(npy_final_path+filename_,np_array)
#

## Additional help and test functoins 

In [ ]:
#Functions for finding the biggest pcd

def count_points_in_pcd(file_path):
    pcd = o3d.io.read_point_cloud(file_path)
    return len(pcd.points)

def find_pcd_with_most_points(folder_path):
    max_points = 0
    max_pcd_file = None

    for file_name in os.listdir(folder_path):
        if file_name.endswith('.pcd'):
            file_path = os.path.join(folder_path, file_name)
            num_points = count_points_in_pcd(file_path)
            if num_points > max_points:
                max_points = num_points
                max_pcd_file = file_path

    return max_pcd_file, max_points

# Replace 'your_folder_path' with the path to your folder containing the PCD files
folder_path = "/home/omar/TUM/Data/cropped/real"

max_pcd_file, max_points = find_pcd_with_most_points(folder_path)
if max_pcd_file:
    print(f'The PCD file with the most points is: {max_pcd_file}')
    print(f'Number of points: {max_points}')
else:
    print('No PCD files found in the specified folder.')


In [ ]:
#unpadding the point clouds
import os
import open3d as o3d
import numpy as np

# Path to the folder containing PCD files
pcd_folder_path = '/home/omar/TUM/Data/reconstructed_cropped/sim_0404_30v004/'

# List all PCD files in the folder
file_list = [f for f in os.listdir(pcd_folder_path) if f.endswith('.pcd')]

# Iterate through each PCD file
for file_name in file_list:
    # Read the point cloud
    pcd = o3d.io.read_point_cloud(os.path.join(pcd_folder_path, file_name))
    
    # Get the numpy array of points
    points = np.asarray(pcd.points)
    
    # Find the indices of non-padded points (non-zero points along the first axis)
    non_padded_indices = np.where(np.any(points != 0, axis=1))[0]
    
    # Extract non-padded points
    non_padded_points = points[non_padded_indices]
    
    # Update the point cloud with non-padded points
    pcd.points = o3d.utility.Vector3dVector(non_padded_points)
    print(len(non_padded_points))  # Print the number of non-padded points
    print(non_padded_points[:5])   # Print the first 5 non-padded points

    # Save the updated point cloud back to file
    o3d.io.write_point_cloud(os.path.join(pcd_folder_path, file_name), pcd)


In [ ]:
# correcting the labels
labels_folder="/home/omar/TUM/data_MA/m1695833/Sim2RealDistributionAlignedDataset/sim/data/label/"
save_folder="/home/omar/TUM/Data/SeedFormer_2602_npy/sim/labels/"
for label in os.listdir(labels_folder):
    file = open(labels_folder+label)
    bbox=file.read()
    bbox_correct=utils.correct_bbox_label(bbox)
    file.close()
    file_write=open(save_folder+label,"w+")
    file_write.write(" ".join(bbox_correct))
    file_write.close()
    print(bbox_correct)
    #print(items)

In [ ]:
#visualization functions for fixing camera position
def print_camera_pose(vis):
    view_control = vis.get_view_control()
    parameters = view_control.convert_to_pinhole_camera_parameters()
    print("Camera Pose:")
    print("Extrinsic:")
    print(parameters.extrinsic)
    print("Intrinsic:")
    print(parameters.intrinsic.intrinsic_matrix)
    return False
def set_camera_pose(vis, extrinsic, intrinsic):
    # Convert extrinsic matrix to 4x4 numpy array
    extrinsic_matrix = np.array(extrinsic)

    # Get the window size from the visualizer
    window_width = vis.get_view_control().convert_to_pinhole_camera_parameters().intrinsic.width
    window_height = vis.get_view_control().convert_to_pinhole_camera_parameters().intrinsic.height

    # Create an Open3D PinholeCameraIntrinsic object with the actual window size
    intrinsic_o3d = o3d.camera.PinholeCameraIntrinsic(
        window_width, window_height,
        intrinsic[0, 0], intrinsic[1, 1],
        intrinsic[0, 2], intrinsic[1, 2]
    )

    # Set camera parameters
    parameters = vis.get_view_control().convert_to_pinhole_camera_parameters()
    parameters.extrinsic = extrinsic_matrix
    parameters.intrinsic = intrinsic_o3d

    vis.get_view_control().convert_from_pinhole_camera_parameters(parameters)

def visualize(point_cloud, bbox_coordinates, extrinsic, intrinsic):
    # Create Open3D point cloud
    vis = o3d.visualization.Visualizer()
    vis.create_window()

    vis.get_render_option().point_size = 3.0
  #  vis.get_render_option().background_color = np.zeros(3)

    pcl = o3d.geometry.PointCloud()
    pcl.points = o3d.utility.Vector3dVector(point_cloud)
    desired_color = [1, 0, 1]  # Red color
    np_colors = np.tile(desired_color, (len(pcl.points), 1))
   # pcl.colors = o3d.utility.Vector3dVector(np_colors)
    # Create Open3D bounding box
    bbox = None
    if bbox_coordinates is not None:
        bbox = o3d.geometry.OrientedBoundingBox(center=bbox_coordinates[:3],
                                                 R=np.eye(3),
                                                 extent=bbox_coordinates[3:6])
        bbox.color = [1, 0, 0]  # Set bbox color to red

    # Add geometries to the visualizer
    vis.clear_geometries()
    vis.add_geometry(pcl)
    #if bbox:
       # vis.add_geometry(bbox)

    # Set initial view
    view_control = vis.get_view_control()
    #vis.poll_events()
    #vis.update_renderer()
  #  set_camera_pose(view_control, extrinsic, intrinsic)
   # vis.register_key_callback(ord("P"), print_camera_pose)

    # Run the visualizer
    vis.run()
    vis.destroy_window()
